## Load data

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
file_list=[]


for filename in os.listdir('./data/output/'):
    if filename.endswith(".pkl"):
        unpickle = './data/output/'+str(filename)
        print(unpickle)
        file_list.append(pd.read_pickle(unpickle))
 
 
MASTER = pd.concat(file_list)
MASTER['time'] = pd.to_datetime(MASTER['time'])

print(MASTER.columns)

./data/output/df2013.pkl
./data/output/df2012.pkl
./data/output/df2010.pkl
./data/output/df2011.pkl
./data/output/df2015.pkl
./data/output/df2014.pkl
./data/output/df2016.pkl
./data/output/df2017.pkl
./data/output/df2019.pkl
./data/output/df2018.pkl
./data/output/df2008.pkl
./data/output/df2020.pkl
./data/output/df2009.pkl


## Show banks

In [ ]:
print(MASTER['InstitutionDescription'].unique())

## Make asset weights for one bank

In [ ]:
# show portfolio weights for absa's credit book
from assets_to_weights import *
years=[str(i) for i in range(2008,2020, 1)]
months=["{:02d}".format(i) for i in range(1,13)]

helper_dict={}

for y in years:
    helper_dict[y]=months
    
BANK = MASTER[MASTER['InstitutionDescription']=='ABSA BANK LTD ']
BANK['Value'] = pd.to_numeric(BANK['Value'])

bank_year_and_month = []
for key in helper_dict.keys():
    for month in helper_dict[key]:
        df_bank_key_month =  BANK[(BANK['TheYear']==key)&(BANK['TheMonth']==month)]
        try:
            bank_year_and_month.append(get_pf_weights_by_bank_29(df_bank_key_month))
        except:
            pass
 
absa=pd.concat(bank_year_and_month) 

# we only need one entry per month - so select column code 7 and itemnumber 2 for example 
absa_weights=absa[(absa['ColumnCode']=='7')&(absa['ItemNumber']=='2')]
absa_weights.index=absa_weights.time

f = plt.figure()
plt.title('ABSA portfolio weights', color='black')
 
for column in absa_weights[absa.columns[-19:-10]]:
    absa_weights[column].plot(legend=column, ax=f.gca())
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

In [ ]:
#make ass

